In [49]:
import torch
import torch.nn as nn
import numpy as np
import math

In [50]:
with open("data/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


In [51]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        x[x<0] *= -1
        x = (x-np.min(x))/(np.max(x) - np.min(x))
        output_array.append(x)
    return np.asarray(output_array)
x_norm = normalise_2darray(all_data)

In [52]:
def train_test_split(input_data, input_labels):
    indices = np.random.permutation(input_data.shape[0])
    split_idx = math.floor(input_data.shape[0] * 0.7)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    train_data, test_data = input_data[train_idx,:], input_data[test_idx,:]
    train_labels, test_labels = input_labels[train_idx], input_labels[test_idx]
    return train_data, test_data, train_labels, test_labels

In [63]:
corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [labels[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

In [64]:
to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)
to_drop

[3,
 4,
 7,
 12,
 13,
 14,
 15,
 16,
 17,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 48]

In [65]:
x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape

(208, 35)

In [68]:
train_data, test_data, train_labels, test_labels = train_test_split(x_norm, labels)
print(train_data.shape, test_data.shape, train_labels.shape, test_data.shape)


(145, 35) (63, 35) (145,) (63, 35)


In [69]:
train_data = torch.tensor(train_data, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.float32).reshape(-1,1)
test_labels = torch.tensor(test_labels, dtype=torch.float32).reshape(-1,1)
print(train_data.shape, test_data.shape, train_labels.shape, test_labels.shape)

torch.Size([145, 35]) torch.Size([63, 35]) torch.Size([145, 1]) torch.Size([63, 1])


In [70]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, input, labels, n_epochs, loss_fn, optimizer, batch_size=10):
        for epoch in range(n_epochs):
            start = 0
            #for i in range(math.floor(input.shape[0]/batch_size)):
            #    X_batch = input[start:(start+batch_size)]
            #    Ybatch = labels[start:(start+batch_size)]
            #    start += batch_size
            #    pass
            prediction = self.forward(input)
            loss = loss_fn(prediction, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch}:\tloss:{loss}")
            
    


In [72]:
model = Network(input_size=35, output_size=1)
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.network.parameters(), lr=0.0025, alpha=0.8)
model.train()
model.fit(input=train_data, labels=train_labels, n_epochs=1000, loss_fn=loss_fn, optimizer=optimizer)

Epoch 0:	loss:0.7048636674880981
Epoch 1:	loss:0.7330915331840515
Epoch 2:	loss:0.6931250691413879
Epoch 3:	loss:0.6885966062545776
Epoch 4:	loss:0.6879522204399109
Epoch 5:	loss:0.6878359317779541
Epoch 6:	loss:0.6877980828285217
Epoch 7:	loss:0.6877853870391846
Epoch 8:	loss:0.6877989172935486
Epoch 9:	loss:0.6879156231880188
Epoch 10:	loss:0.68852299451828
Epoch 11:	loss:0.6916123628616333
Epoch 12:	loss:0.7001680135726929
Epoch 13:	loss:0.7000827193260193
Epoch 14:	loss:0.6922555565834045
Epoch 15:	loss:0.6886746883392334
Epoch 16:	loss:0.6869450211524963
Epoch 17:	loss:0.6852424144744873
Epoch 18:	loss:0.6835047006607056
Epoch 19:	loss:0.6855544447898865
Epoch 20:	loss:0.69038987159729
Epoch 21:	loss:0.6874575018882751
Epoch 22:	loss:0.6728107333183289
Epoch 23:	loss:0.6651995182037354
Epoch 24:	loss:0.6607453227043152
Epoch 25:	loss:0.669995903968811
Epoch 26:	loss:0.6651620864868164
Epoch 27:	loss:0.6630240678787231
Epoch 28:	loss:0.6473339200019836
Epoch 29:	loss:0.640166103839

In [60]:
model.eval()
print(model.forward(test_data))
print(test_labels)

tensor([[0.7088],
        [0.0435],
        [0.0397],
        [0.7088],
        [0.7087],
        [0.0652],
        [0.0393],
        [0.7085],
        [0.7087],
        [0.7085],
        [0.7085],
        [0.7056],
        [0.7018],
        [0.7068],
        [0.7086],
        [0.7084],
        [0.7083],
        [0.7088],
        [0.6996],
        [0.7088],
        [0.7089],
        [0.7086],
        [0.0418],
        [0.7086],
        [0.7012],
        [0.7089],
        [0.7074],
        [0.7088],
        [0.7038],
        [0.7072],
        [0.0455],
        [0.6789],
        [0.7073],
        [0.0512],
        [0.6948],
        [0.7086],
        [0.7088],
        [0.7089],
        [0.7079],
        [0.7089],
        [0.2311],
        [0.7076],
        [0.4286],
        [0.0797],
        [0.7088],
        [0.7084],
        [0.0433],
        [0.7084],
        [0.0476],
        [0.7057],
        [0.6983],
        [0.4122],
        [0.7075],
        [0.5942],
        [0.7037],
        [0